# Comparison of two indices 

For this analysis, I analyse two different indices, both small cap. The US S&P 600 Small Caps and the Eurostoxx Small, both from the iShares ETF page. Both ETFs are optimized in their replication method, meaning that they try to follow the devlopment of their benchmark not by replicating the composition exactly but in a similar fashion. Companies in these ETFs therefore are not exactly the same as in the original indices. 

According to finanzen.net, the closing of the change between EUR and USD was 1.1986. 

## General information 

In [103]:
import numpy as np
import pandas as pd
from IPython.display import Markdown as md

In [110]:
#Read in the datasets and save as Pandas Dataframe 
sp600small=pd.read_csv('IUS3_holdings.csv', engine='python', skiprows=1, encoding='UTF8', thousands='.', decimal=',')
eurostoxxsmall=pd.read_csv('IQQS_holdings.csv', engine='python', skiprows=1, encoding='UTF8', thousands='.', decimal=',')

sp600small=pd.DataFrame(sp600small)
eurostoxxsmall=pd.DataFrame(eurostoxxsmall)

exchange_rate_EUR_USD=1.1986

In [111]:
md(f"The S&P 600 Small ETF has {len(sp600small)} items and a represented market value of. The Eurostoxx Small ETF is much smaller with {len(eurostoxxsmall)} items only.")

The S&P 600 Small ETF has 606 items and a represented market value of. The Eurostoxx Small ETF is much smaller with 95 items only.

In [106]:
market_value_EU=eurostoxxsmall.groupby(['Marktwährung']).agg({'Marktwert': sum})
print(market_value_EU.loc['EUR'])

market_value_US=sp600small.groupby(['Marktwährung']).agg({'Marktwert': sum})
print(market_value_US.loc['USD'])

Marktwert    530860911
Name: EUR, dtype: int64
Marktwert    1400662760
Name: USD, dtype: int64


In [113]:
round(market_value_EU.loc['EUR']*exchange_rate_EUR_USD/market_value_US.loc['USD'], 2)

Marktwert    0.45
dtype: float64

The European index represents not even half the marketvalue of the US American index. It is nevertheless impressive: With the smaller index having 1/6 of the number of companies. The average marketvalue of a company in the index (including the cash positions) of Europe is therefore nearly 4 times the size of one of the US American one. 

For investors, each company in the European index can be thought of as more valuable than the ones in the American index. 

In [108]:
print('S&P 600 Small: ', round(sp600small['Marktwert'].mean()), 'USD')
print('Eurostoxx Small: ', round(eurostoxxsmall['Marktwert'].mean()*exchange_rate_EUR_USD), 'USD')

S&P 600 Small:  2311625 USD
Eurostoxx Small:  6699994 USD


## Sector analysis 

In [11]:
share_by_sector=sp600small.groupby('Sektor', as_index=False).agg({'Gewichtung (%)': "sum", 
                                                                 'Marktwert':sum})

In [12]:
share_by_sector.sort_values(by=['Gewichtung (%)'], ascending=False)

,Sektor,Gewichtung (%),Marktwert
6,Industrie,17.88,250302439
2,Financials,17.52,245486339
11,Zyklische Konsumgüter,16.17,226229927
4,IT,13.29,186051431
3,Gesundheitsversorgung,11.22,157036112
5,Immobilien,7.27,101858931
8,Materialien,4.90,68439879
1,Energie,4.19,58571915
9,Nichtzyklische Konsumgüter,3.83,53947893
7,Kommunikation,2.10,29724378


In [13]:
share_by_sector_EU=eurostoxxsmall.groupby('Sektor', as_index=False).agg({'Gewichtung (%)': "sum",
                                                                        'Marktwert':sum})
share_by_sector_EU.sort_values(by=['Gewichtung (%)'], ascending=False)

,Sektor,Gewichtung (%),Marktwert
6,Industrie,17.34,92028491
5,Immobilien,14.05,74579854
2,Financials,13.44,71368098
7,Kommunikation,9.12,48471758
10,Versorger,8.11,43056637
8,Materialien,7.42,39438584
3,Gesundheitsversorgung,7.23,38339991
4,IT,6.72,35656092
11,Zyklische Konsumgüter,5.85,31108256
9,Nichtzyklische Konsumgüter,5.69,30247943


In [14]:
shares_sector_indices=pd.merge(share_by_sector_EU, share_by_sector, on='Sektor', suffixes=('_EU','_US'))
shares_sector_indices['Difference in %points']=shares_sector_indices['Gewichtung (%)_EU']-shares_sector_indices['Gewichtung (%)_US']
shares_sector_indices['Difference in Marketvalue']=shares_sector_indices['Marktwert_EU']-shares_sector_indices['Marktwert_US']
shares_sector_indices.sort_values(by='Gewichtung (%)_EU', ascending=False)

,Sektor,Gewichtung (%)_EU,Marktwert_EU,Gewichtung (%)_US,Marktwert_US,Difference in %points,Difference in Marketvalue
6,Industrie,17.34,92028491,17.88,250302439,-0.54,-158273948
5,Immobilien,14.05,74579854,7.27,101858931,6.78,-27279077
2,Financials,13.44,71368098,17.52,245486339,-4.08,-174118241
7,Kommunikation,9.12,48471758,2.10,29724378,7.02,18747380
10,Versorger,8.11,43056637,1.38,19229321,6.73,23827316
8,Materialien,7.42,39438584,4.90,68439879,2.52,-29001295
3,Gesundheitsversorgung,7.23,38339991,11.22,157036112,-3.99,-118696121
4,IT,6.72,35656092,13.29,186051431,-6.57,-150395339
11,Zyklische Konsumgüter,5.85,31108256,16.17,226229927,-10.32,-195121671
9,Nichtzyklische Konsumgüter,5.69,30247943,3.83,53947893,1.86,-23699950


In [15]:
more_important_US=shares_sector_indices['Difference in %points']<0
more_important_EU=shares_sector_indices['Difference in %points']>0

In [16]:
shares_sector_indices[more_important_US].sort_values(by='Difference in %points', ascending=True)

,Sektor,Gewichtung (%)_EU,Marktwert_EU,Gewichtung (%)_US,Marktwert_US,Difference in %points,Difference in Marketvalue
11,Zyklische Konsumgüter,5.85,31108256,16.17,226229927,-10.32,-195121671
4,IT,6.72,35656092,13.29,186051431,-6.57,-150395339
2,Financials,13.44,71368098,17.52,245486339,-4.08,-174118241
3,Gesundheitsversorgung,7.23,38339991,11.22,157036112,-3.99,-118696121
6,Industrie,17.34,92028491,17.88,250302439,-0.54,-158273948


In [17]:
shares_sector_indices[more_important_EU].sort_values(by='Difference in %points', ascending=False)

,Sektor,Gewichtung (%)_EU,Marktwert_EU,Gewichtung (%)_US,Marktwert_US,Difference in %points,Difference in Marketvalue
7,Kommunikation,9.12,48471758,2.10,29724378,7.02,18747380
5,Immobilien,14.05,74579854,7.27,101858931,6.78,-27279077
10,Versorger,8.11,43056637,1.38,19229321,6.73,23827316
8,Materialien,7.42,39438584,4.90,68439879,2.52,-29001295
9,Nichtzyklische Konsumgüter,5.69,30247943,3.83,53947893,1.86,-23699950
1,Energie,4.54,24118065,4.19,58571915,0.35,-34453850
0,Cash und/oder Derivate,0.50,2621999,0.28,3966105,0.22,-1344106


There are important differences in the composition of the indices: While the US set more importance (in terms of share of the index) and consumer goods, IT, Financials and Health companies, the European index has an comparable overweight of communications, utilities, real estate, metails and non-cyclical consumer goods. Small differences are for the sectors of industry, energy and the cash reserve. 

TODO: Currency conversion 

What are the most important sector, according to market value?

In [18]:
share_by_stockexchange=sp600small.groupby('Börse', as_index=False).agg({'Gewichtung (%)': "sum"})
share_by_stockexchange.sort_values(by=['Gewichtung (%)'], ascending=False)

,Börse,Gewichtung (%)
3,New York Stock Exchange Inc.,50.05
2,NASDAQ,49.35
4,Nyse Mkt Llc,0.35
0,-,0.28
1,Chicago Mercantile Exchange,0.00


In [19]:
sp600small.groupby('Anlageklasse', as_index=False).agg({'Gewichtung (%)': "sum"})

,Anlageklasse,Gewichtung (%)
0,Aktien,99.75
1,Cash Collateral and Margins,0.00
2,Futures,0.00
3,Geldmarkt,0.28


In [20]:
sp600small.groupby('Marktwährung', as_index=False).agg({'Gewichtung (%)': "sum"})

,Marktwährung,Gewichtung (%)
0,EUR,0.00
1,GBP,0.01
2,USD,100.02


Differences by countries 

With the S&P 600 Small being an US American index, it is not surprising that nearly all the companies in the index have the US as their country of domicile. Only one is based in the UK: 

In [21]:
sp600small.groupby('Standort').agg({'Gewichtung (%)': sum}).sort_values(by='Gewichtung (%)', ascending=False)

,Gewichtung (%)
Standort,
Vereinigte Staaten,100.02
Vereinigtes Königreich,0.01
-,0.00
Europäische Union,0.00


In [22]:
uk_based=sp600small['Standort'].str.match('Vereinigtes Königreich')
sp600small[uk_based]

,Emittententicker,Name,Anlageklasse,Gewichtung (%),Kurs,Nominale,Marktwert,Nominalwert,Sektor,ISIN,Börse,Standort,Marktwährung
599,GBP,GBP CASH,Geldmarkt,0.01,139.69,130220,181904,181904.28,Cash und/oder Derivate,-,-,Vereinigtes Königreich,GBP


The only non-American position in the ETF is Cash or Derivates in the United Kingdom. This makes up for only 0.01% of the ETF volume. 

This is different for the European Small Cap ETF. This covers several European countries, mainly from Western and Northern Europe. There are also small positions in the US, the UK and the EU directly. All of them are Cash and/or Derivates. 

In [23]:
eurostoxxsmall.groupby('Standort').agg({'Gewichtung (%)': sum}).sort_values(by='Gewichtung (%)', ascending=False)

,Gewichtung (%)
Standort,
Frankreich,25.72
Deutschland,22.73
Italien,10.29
Belgien,9.06
Niederlande,8.20
Österreich,7.02
Spanien,5.84
Finnland,5.31
Irland,2.13


In [24]:
external_based=(eurostoxxsmall['Standort'].str.match('Vereinigtes Königreich'))|(eurostoxxsmall['Standort'].str.match('Europäische Union'))|(eurostoxxsmall['Standort'].str.match('Vereinigte Staaten'))
eurostoxxsmall[external_based]

,Emittententicker,Name,Anlageklasse,Gewichtung (%),Kurs,Nominale,Marktwert,Nominalwert,Sektor,ISIN,Börse,Standort,Marktwährung
39,FTI,TECHNIPFMC PLC,Aktien,1.18,7.60,822249,6250737,6250736.90,Energie,GB00BDSFG982,Nyse Euronext - Euronext Paris,Vereinigte Staaten,EUR
89,EUR,EUR CASH,Geldmarkt,0.34,100.00,1803418,1803418,1803417.97,Cash und/oder Derivate,-,-,Europäische Union,EUR
91,BZFUT,CASH COLLATERAL EUR BZFUT,Cash Collateral and Margins,0.04,100.00,216440,216440,216440.00,Cash und/oder Derivate,-,-,Europäische Union,EUR
92,GBP,GBP CASH,Geldmarkt,0.03,116.79,113781,132888,132887.61,Cash und/oder Derivate,-,-,Vereinigtes Königreich,GBP
93,USD,USD CASH,Geldmarkt,0.01,83.61,50197,41969,41968.68,Cash und/oder Derivate,-,-,Vereinigte Staaten,USD
94,VGH1,EURO STOXX 50 MAR 21,Futures,0.00,3844.00,64,0,2460160.00,Cash und/oder Derivate,DE000C31Y1X2,Eurex Deutschland,Europäische Union,EUR


Are there companies that are the same?

In [25]:
common_companies=pd.merge(left=eurostoxxsmall, right=sp600small, on='Name', how='inner')

common_companies

,Emittententicker_x,Name,Anlageklasse_x,Gewichtung (%)_x,Kurs_x,Nominale_x,Marktwert_x,Nominalwert_x,Sektor_x,ISIN_x,...,Gewichtung (%)_y,Kurs_y,Nominale_y,Marktwert_y,Nominalwert_y,Sektor_y,ISIN_y,Börse_y,Standort_y,Marktwährung_y
0,EUR,EUR CASH,Geldmarkt,0.34,100.00,1803418,1803418,1803417.97,Cash und/oder Derivate,-,...,0.00,119.60,5,6,5.86,Cash und/oder Derivate,-,-,Europäische Union,EUR
1,GBP,GBP CASH,Geldmarkt,0.03,116.79,113781,132888,132887.61,Cash und/oder Derivate,-,...,0.01,139.69,130220,181904,181904.28,Cash und/oder Derivate,-,-,Vereinigtes Königreich,GBP
2,USD,USD CASH,Geldmarkt,0.01,83.61,50197,41969,41968.68,Cash und/oder Derivate,-,...,0.27,100.00,3718195,3718195,3718195.02,Cash und/oder Derivate,-,-,Vereinigte Staaten,USD


The only shared positions are those of cash in various currencies: USD, EUR and GBP. 